# Warmup: Counter. 

Count how many times each element in a list occurs.

```
[1, 3, 2, 1, 5, 3, 5, 1, 4] ⇒

    1: 3 times
    2: 1 time
    3: 2 times
    4: 1 time
    5: 2 times
```


In [15]:
def counter(lst):
    dictionary = {}
    for i in lst:
        if i in dictionary :
            dictionary[i]+=1
        else:
            dictionary[i]=1
    res =''
    for k,v in sorted(dictionary.items(), key = lambda item: item[0]):
        if v <= 1:
            res += f"{k}: {v} time\n"
        else:
            res += f"{k}: {v} times\n"
    return res

#Test
lst1 = [1,3,2,1,5,3,5,1,4]
print(counter(lst1))
    

1: 3 times
2: 1 time
3: 2 times
4: 1 time
5: 2 times



# Safe dict reading

define a function `safe_dict(d, k)` that takes in a python dict `d` and a key `k` and makes it safe to read even with keys that aren't in the dictionary. If you try to read from the dictionary with a bad key, it should return 0 instead.

```
d = {1 : 2, 3 : 4}
safe_dict(d, 1) -> 2
safe_dict(d, 'cat') -> 0
```

In [16]:
def safe_dict(d, k):
    return d.get(k,0)

#test
d = {1 : 2, 3 : 4}  
print(safe_dict(d,1))
print(safe_dict(d, 'cat'))

2
0


# File Reading: Hamlet Exercises

Open `hamlet.txt` in the `data` folder

### 1. Mentionned Hamlet

How many times is hamlet mentioned in the book?

Use python and line iteration to count it up

In [17]:
def replace_chars(text):
    '''
    Removing special characters and punctuations.
    '''
    text = text.replace('’s','').replace('-',' ').replace('\ufeff','').replace('—', ' ' )
    chars = ",.!?[]_:;‘’"
    for c in chars:
        text = text.replace(c,'')
    return text
        
with open("data/hamlet.txt", "r",encoding='utf8') as hamlet:
    
    count_hamlet = 0
    
    for line in hamlet.readlines():
        line = replace_chars(line).split()
        
        for word in line:
            if word.lower() == 'hamlet':
                count_hamlet +=1
                
    print(f"Answer: Hamlet is mentioned {count_hamlet} times in the book.")



Answer: Hamlet is mentioned 474 times in the book.


### 2. File Reading as a .py program

Make a python file that defines a function that counts the number of times hamlet is mentionned using the code in the previous exercise.

Then import it in your notebook and call it here.

In [18]:
with open("file_reader.py", 'w',encoding='utf8') as f:
    f.write(
'''def replace_chars(text):
    text = text.replace('’s','').replace('-',' ').replace('\ufeff','').replace('—', ' ' )
    chars = ",.!?[]_:;‘’"
    for c in chars:
        text = text.replace(c,'')
    return text

def file_reader(file):

    with open(file, "r",encoding='utf8') as hamlet:
        count_hamlet = 0
        for line in hamlet.readlines():
            line = replace_chars(line).split()

            for word in line:
                if word.lower() == 'hamlet':
                    count_hamlet +=1

        string = f"Answer: Hamlet is mentioned {count_hamlet} times in the book."
        return string
''')



In [19]:
import file_reader as fr
file = "data/hamlet.txt"
fr.file_reader(file)

'Answer: Hamlet is mentioned 474 times in the book.'

### 3. Unique words in hamlet

Write a program that counts the unique words in hamlet.

In [20]:
def unique_word(file):
    with open(file, "r", encoding='utf8') as hamlet:
        words_in_hamlet =[]
        for line in hamlet.readlines():
            line = replace_chars(line).lower().split()
            for word in line:
                words_in_hamlet.append(word)
        unique_words = set(words_in_hamlet)     
        num_unique_words = len(unique_words)
        f_name = file.split('/')[1][:-4]
        return f"Answer : After removing all whitespaces and punctuations, there are around {num_unique_words} unique \
words in {f_name.title()}."
    
f = "data/hamlet.txt"
print(unique_word(f))


Answer : After removing all whitespaces and punctuations, there are around 4688 unique words in Hamlet.


# File Reading 2: A Python library.

In the `data` folder, you will find a folder called `csrgraph` which is a python library.

### 1. File count

Count the `py` files in the library using the `os` package

In [21]:
import os

relative_path = '.\data\csrgraph'
num_py_files = len([file for file in os.listdir(relative_path) if file.endswith('.py')])
print(f"Answer: There are {num_py_files} .py in the csrgraph library.")


Answer: There are 8 .py in the csrgraph library.


### 2. For the following packages, count the number of files that import them:

- pandas 

- numpy

- numba

In [22]:
py_files = [file for file in os.listdir(relative_path) if file.endswith('.py')]
paths=[]
for py_file in py_files:
    paths.append(os.path.join('data','csrgraph', py_file))

count_lib_imports={} 
for i in range(len(paths)):
    with open(paths[i], 'r') as f:
        text = f.read()
        for lib_import in ['import pandas', 'import numpy', 'import numba']:
            if text.find(lib_import)!=-1: 
                key = lib_import.split()[1]

                if key in count_lib_imports:
                    count_lib_imports[key] +=1
                else:
                    count_lib_imports[key] = 1
                    
print(count_lib_imports)

{'numpy': 6, 'numba': 5, 'pandas': 4}


# First NLP Program: IDF

Given a list of words, the the inverse document frequency (IDF) is a basic statistic of the amount of information of each word in the text.

The IDF formulat is:

$$IDF(w) = ln(\dfrac{N}{1 + n(w)})$$

Where:

- $w$ is the token (unique word),
- $n(w)$ is the number of documents that $w$ occurs in,
- $N$ is the total number of documents

Write a function, `idf(docs)` that takes in a list of lists of words and returns a dictionary  `word -> idf score`

Example:

```
IDF([['interview', 'questions'], ['interview', 'answers']]) -> {'questions': 0.0, 
                                                                'interview': -0.4, 
                                                                'answers': 0.0}


```

In [23]:
import numpy as np
def IDF(docs):
    '''
    Function that takes in a lists of words and returns a dictionary containing the idf score for each words.
    '''
    N = len(docs) #Total number of documents

    #Find all unique words in the collection of documents
    unique_words = set()
    for i in range(N):
        unique_words = unique_words.union(set(docs[i]))
    unique_words = set(unique_words)
    
    #Store in a dictionary the number of documents in which a word appears
    num_doc_with_word = {}
    for unique_word in unique_words:
        for doc in docs:
            if unique_word in doc:
                if unique_word in num_doc_with_word:
                    num_doc_with_word[unique_word]+=1
                else:
                    num_doc_with_word[unique_word]=1 
                    
    #Calculate for each unique word in the collection its idf
    idf = {}
    for k,v in num_doc_with_word.items():
        idf[k] = round(np.log(N/(1+v)),1)
        
    return idf
                
#Test

IDF([['interview', 'questions'], ['interview', 'answers']])   


{'interview': -0.4, 'answers': 0.0, 'questions': 0.0}

# Stretch Goal: IDF on Hamlet


The TF-IDF score is a commonly used statistic for the importance of words. Its $\frac{TF}{IDF}$ where TF is the "term frequency" (eg. how often the words happens in the document).

Calculate the TF-IDF dictionary on the Hamlet book.

What's the TF-IDF of "Hamlet"?

What's the word with the highest TF-IDF in the book?



In [24]:
from collections import Counter
file = 'data/hamlet.txt'

with open(file, 'r', encoding = 'utf8') as hamlet:
    input_list = []
    words_count={}
    tf_idf = {}
    
    #Divide text by lines, create list of word for each line and store result in list
    for line in hamlet.readlines():
        words_list = replace_chars(line).lower().split()
        input_list.append(words_list)
        
    #Create idf dictionary
    idf_dict = IDF(input_list)
    
    #Find frequency of each word in a line and multiply with idf for the given word
    tf_idf_dict = {}
    for i in range(len(input_list)):
        tf_idf = Counter(input_list[i]) 
        num_words = len(input_list[i])

        for k,v in tf_idf.items():
            tf_idf[k] = v/num_words*idf_dict[k]
            
        #stores tf_idf for the line in a dictionary
        tf_idf_dict[i] = tf_idf
        

    


In [25]:
#Calculate the TF-IDF dictionary on the Hamlet book.
#Print tf-idf dictionary on Hamlet book
for k1,v1 in tf_idf_dict.items():
    print(f"Line : {k1}\n")
    for k2, v2 in v1.items():
          print(f"{k2} : {v2:.2f}")
    print('\n')

Line : 0

the : 0.29
tragedy : 1.06
of : 0.66
hamlet : 0.39
prince : 0.99
denmark : 0.77


Line : 1



Line : 2

contents : 7.70


Line : 3



Line : 4

act : 2.80
i : 1.25


Line : 5

scene : 0.62
i : 0.31
elsinore : 0.79
a : 0.33
platform : 0.86
before : 0.71
the : 0.25
castle : 0.68


Line : 6

scene : 0.50
ii : 0.62
elsinore : 0.63
a : 0.26
room : 0.54
of : 0.23
state : 0.60
in : 0.27
the : 0.20
castle : 0.54


Line : 7

scene : 0.71
iii : 0.94
a : 0.37
room : 0.77
in : 0.39
polonius : 0.57
house : 0.93


Line : 8

scene : 1.25
iv : 1.65
the : 0.50
platform : 1.73


Line : 9

scene : 0.56
v : 0.77
a : 0.29
more : 0.47
remote : 0.86
part : 0.61
of : 0.26
the : 0.22
castle : 0.60


Line : 10



Line : 11

act : 2.80
ii : 3.10


Line : 12

scene : 0.71
i : 0.36
a : 0.37
room : 0.77
in : 0.39
polonius : 0.57
house : 0.93


Line : 13

scene : 0.71
ii : 0.89
a : 0.37
room : 0.77
in : 0.39
the : 0.29
castle : 0.77


Line : 14



Line : 15

act : 2.80
iii : 3.30


Line : 16

scene : 0.71
i

bodes : 1.01
some : 0.60
strange : 0.81
eruption : 1.01
to : 0.29
our : 0.51
state : 0.75


Line : 250



Line : 251

marcellus : 4.90


Line : 252

good : 0.42
now : 0.42
sit : 0.66
down : 0.54
and : 0.20
tell : 0.50
me : 0.34
he : 0.35
that : 0.28
knows : 0.74


Line : 253

why : 0.59
this : 0.39
same : 0.78
strict : 0.96
and : 0.25
most : 0.55
observant : 1.01
watch : 0.75


Line : 254

so : 0.44
nightly : 1.01
toils : 1.01
the : 0.50
subject : 0.93
of : 0.29
land : 0.88


Line : 255

and : 0.25
why : 0.59
such : 0.59
daily : 1.01
cast : 0.86
of : 0.29
brazen : 1.01
cannon : 0.88


Line : 256

and : 0.29
foreign : 1.16
mart : 1.16
for : 0.47
implements : 1.16
of : 0.33
war : 1.10


Line : 257

why : 0.59
such : 0.59
impress : 1.01
of : 0.29
shipwrights : 1.01
whose : 0.66
sore : 0.93
task : 1.01


Line : 258

does : 0.69
not : 0.39
divide : 0.96
the : 0.50
sunday : 1.01
from : 0.54
week : 1.01


Line : 259

what : 0.44
might : 0.66
be : 0.44
toward : 0.86
that : 0.35
this : 0.39
swe

to : 0.26
business : 0.70
with : 0.36
the : 0.44
king : 0.39
more : 0.47
than : 0.54
scope : 0.82


Line : 443

of : 0.46
these : 0.98
dilated : 1.62
articles : 1.62
allow : 1.62


Line : 444

farewell : 0.74
and : 0.25
let : 0.51
your : 0.85
haste : 0.76
commend : 0.90
duty : 0.78


Line : 445



Line : 446

cornelius : 2.23
and : 0.67
voltemand : 2.17


Line : 447

in : 0.27
that : 0.28
and : 0.20
all : 0.40
things : 0.63
will : 0.37
we : 0.39
show : 0.57
our : 0.41
duty : 0.62


Line : 448



Line : 449

king : 3.50


Line : 450

we : 0.65
doubt : 1.05
it : 0.48
nothing : 0.88
heartily : 1.23
farewell : 0.98


Line : 451



Line : 452

exeunt : 1.30
voltemand : 1.62
and : 0.50
cornelius : 1.68


Line : 453



Line : 454

and : 0.25
now : 0.53
laertes : 0.51
what : 0.44
the : 0.25
news : 0.80
with : 0.40
you : 0.33


Line : 455

you : 0.29
told : 0.78
us : 0.51
of : 0.26
some : 0.53
suit : 0.82
what : 0.39
ist : 0.66
laertes : 0.46


Line : 456

you : 0.33
cannot : 0.68
speak : 0.59




Line : 582

seem : 0.71
to : 0.26
me : 0.38
all : 0.44
the : 0.22
uses : 0.82
of : 0.26
this : 0.34
world : 0.61


Line : 583

fie : 1.75
ont : 0.81
oh : 0.93
tis : 0.56
an : 0.61
unweeded : 1.01
garden : 1.01


Line : 584

that : 0.28
grows : 0.70
to : 0.23
seed : 0.81
things : 0.63
rank : 0.67
and : 0.20
gross : 0.74
in : 0.27
nature : 0.54


Line : 585

possess : 0.90
it : 0.64
merely : 0.86
that : 0.31
should : 0.53
come : 0.47
to : 0.26
this : 0.34


Line : 586

but : 0.32
two : 1.14
months : 0.72
dead : 0.53
nay : 0.55
not : 0.62
so : 0.35
much : 0.49


Line : 587

so : 0.44
excellent : 0.80
a : 0.33
king : 0.44
that : 0.35
was : 0.55
to : 0.29
this : 0.39


Line : 588

hyperion : 0.86
to : 0.51
a : 0.29
satyr : 0.90
so : 0.39
loving : 0.82
my : 0.29
mother : 0.56


Line : 589

that : 0.31
he : 0.39
might : 0.59
not : 0.34
beteem : 0.90
the : 0.22
winds : 0.82
of : 0.26
heaven : 0.56


Line : 590

visit : 0.88
her : 0.55
face : 0.79
too : 0.61
roughly : 0.96
heaven : 0.62
and :

Line : 796



Line : 797

marcellus : 1.63
and : 0.67
barnardo : 1.80


Line : 798

longer : 7.00


Line : 799



Line : 800

horatio : 3.80


Line : 801

not : 0.78
when : 1.20
i : 0.62
sawt : 2.02


Line : 802



Line : 803

hamlet : 2.70


Line : 804

his : 0.64
beard : 1.38
was : 0.88
grizzled : 1.62
no : 0.78


Line : 805



Line : 806

horatio : 3.80


Line : 807

it : 0.58
was : 0.44
as : 0.35
i : 0.25
have : 0.36
seen : 0.57
in : 0.27
his : 0.32
life : 0.53


Line : 808

a : 0.87
sable : 2.57
silverd : 2.70


Line : 809



Line : 810

hamlet : 2.70


Line : 811

i : 0.62
will : 0.93
watch : 1.50
tonight : 1.57


Line : 812

perchance : 1.75
twill : 1.85
walk : 1.62
again : 1.32


Line : 813



Line : 814

horatio : 3.80


Line : 815

i : 0.50
warrant : 1.40
you : 0.52
it : 0.58
will : 0.74


Line : 816



Line : 817

hamlet : 2.70


Line : 818

if : 0.59
it : 0.41
assume : 1.06
my : 0.37
noble : 0.84
father : 0.66
person : 1.03


Line : 819

ill : 0.51
speak : 0.52
to : 0.26
it

...

till : 0.72
our : 0.51
ground : 0.80


Line : 5888

singeing : 1.16
his : 0.46
pate : 1.03
against : 0.80
the : 0.29
burning : 1.10
zone : 1.16


Line : 5889

make : 0.53
ossa : 0.90
like : 0.49
a : 0.29
wart : 0.90
nay : 0.61
an : 0.54
thoult : 0.86
mouth : 0.77


Line : 5890

ill : 0.77
rant : 1.35
as : 1.17
well : 0.73
thou : 0.70


Line : 5891



Line : 5892

queen : 4.00


Line : 5893

this : 0.78
is : 0.75
mere : 1.85
madness : 1.43


Line : 5894

and : 0.22
thus : 0.57
awhile : 0.71
the : 0.22
fit : 0.69
will : 0.41
work : 0.70
on : 0.44
him : 0.40


Line : 5895

anon : 0.99
as : 1.00
patient : 1.06
the : 0.29
female : 1.16
dove : 1.10


Line : 5896

when : 0.69
that : 0.40
her : 0.63
golden : 1.06
couplets : 1.16
are : 0.57
disclosd : 1.10


Line : 5897

his : 0.64
silence : 1.40
will : 0.74
sit : 1.32
drooping : 1.62


Line : 5898



Line : 5899

hamlet : 2.70


Line : 5900

hear : 1.77
you : 0.87
sir : 1.50


Line : 5901

what : 0.39
is : 0.33
the : 0.22
reason : 0.68
that : 

you : 0.52
from : 0.86
his : 0.64
majesty : 1.20


Line : 6073



Line : 6074

hamlet : 2.70


Line : 6075

i : 0.18
will : 0.26
receive : 0.48
it : 0.21
with : 0.23
all : 0.29
diligence : 0.58
of : 0.16
spirit : 0.44
put : 0.40
your : 0.24
bonnet : 0.58
to : 0.16
his : 0.23


Line : 6076

right : 1.07
use : 1.00
tis : 0.75
for : 0.55
the : 0.33
head : 0.92


Line : 6077



Line : 6078

osric : 5.30


Line : 6079

i : 0.36
thank : 0.93
your : 0.49
lordship : 0.94
tis : 0.64
very : 0.66
hot : 0.99


Line : 6080



Line : 6081

hamlet : 2.70


Line : 6082

no : 0.39
believe : 0.58
me : 0.34
tis : 0.45
very : 0.46
cold : 0.69
the : 0.20
wind : 0.63
is : 0.30
northerly : 0.81


Line : 6083



Line : 6084

osric : 5.30


Line : 6085

it : 0.41
is : 0.43
indifferent : 1.06
cold : 0.99
my : 0.37
lord : 0.49
indeed : 0.77


Line : 6086



Line : 6087

hamlet : 2.70


Line : 6088

methinks : 0.70
it : 0.29
is : 0.30
very : 0.46
sultry : 0.77
and : 0.20
hot : 0.69
for : 0.33
my : 0.26
complexion

will : 0.53
lose : 0.96
this : 0.44
wager : 0.99
my : 0.37
lord : 0.49


Line : 6266



Line : 6267

hamlet : 2.70


Line : 6268

i : 0.33
do : 0.25
not : 0.21
think : 0.33
so : 0.23
since : 0.39
he : 0.23
went : 0.47
into : 0.35
france : 0.47
have : 0.24
been : 0.37
in : 0.18
continual : 0.54


Line : 6269

practice : 0.51
i : 0.18
shall : 0.29
win : 0.53
at : 0.31
the : 0.14
odds : 0.53
but : 0.23
thou : 0.30
wouldst : 0.49
not : 0.22
think : 0.35
how : 0.31
ill : 0.33


Line : 6270

all : 0.40
here : 0.47
about : 0.59
my : 0.26
heart : 0.54
but : 0.32
it : 0.29
is : 0.30
no : 0.39
matter : 0.55


Line : 6271



Line : 6272

horatio : 3.80


Line : 6273

nay : 1.38
good : 1.05
my : 0.65
lord : 0.85


Line : 6274



Line : 6275

hamlet : 2.70


Line : 6276

it : 0.39
is : 0.40
but : 0.43
foolery : 0.54
such : 0.31
a : 0.17
kind : 0.41
of : 0.15
gain : 0.49
giving : 0.48
as : 0.23
would : 0.30


Line : 6277

perhaps : 1.75
trouble : 1.93
a : 0.65
woman : 1.65


Line : 6278



Line : 62



Line : 6478

laertes : 0.41
wounds : 0.77
hamlet : 0.27
then : 0.47
in : 0.27
scuffling : 0.81
they : 0.42
change : 0.72
rapiers : 0.74
and : 0.20


Line : 6479

hamlet : 0.90
wounds : 2.57
laertes : 1.37


Line : 6480



Line : 6481

king : 3.50


Line : 6482

part : 1.10
them : 0.90
they : 0.84
are : 0.80
incensd : 1.54


Line : 6483



Line : 6484

hamlet : 2.70


Line : 6485

nay : 1.83
come : 1.40
again : 1.77


Line : 6486



Line : 6487

the : 0.67
queen : 1.33
falls : 2.17


Line : 6488



Line : 6489

osric : 5.30


Line : 6490

look : 0.87
to : 0.38
the : 0.33
queen : 0.67
there : 0.72
ho : 1.03


Line : 6491



Line : 6492

horatio : 3.80


Line : 6493

they : 0.42
bleed : 0.77
on : 0.40
both : 0.54
sides : 0.74
how : 0.43
is : 0.30
it : 0.29
my : 0.26
lord : 0.34


Line : 6494



Line : 6495

osric : 5.30


Line : 6496

how : 1.43
ist : 1.97
laertes : 1.37


Line : 6497



Line : 6498

laertes : 4.10


Line : 6499

why : 0.52
as : 0.39
a : 0.29
woodcock : 0.90
to : 0.26
m

invite : 0.90
me : 0.38


Line : 6657



Line : 6658

horatio : 3.80


Line : 6659

of : 0.26
that : 0.31
i : 0.28
shall : 0.46
have : 0.40
also : 0.90
cause : 0.66
to : 0.26
speak : 0.52


Line : 6660

and : 0.20
from : 0.43
his : 0.32
mouth : 0.69
whose : 0.53
voice : 0.62
will : 0.37
draw : 0.65
on : 0.40
more : 0.42


Line : 6661

but : 0.46
let : 0.59
this : 0.44
same : 0.89
be : 0.50
presently : 1.00
performd : 1.10


Line : 6662

even : 0.62
while : 0.69
men : 0.68
minds : 0.86
are : 0.44
wild : 0.80
lest : 0.80
more : 0.47
mischance : 0.86


Line : 6663

on : 0.80
plots : 1.54
and : 0.40
errors : 1.62
happen : 1.62


Line : 6664



Line : 6665

fortinbras : 5.70


Line : 6666

let : 1.37
four : 2.33
captains : 2.70


Line : 6667

bear : 0.76
hamlet : 0.34
like : 0.55
a : 0.33
soldier : 0.90
to : 0.29
the : 0.25
stage : 0.90


Line : 6668

for : 0.37
he : 0.78
was : 0.49
likely : 0.86
had : 0.56
been : 0.61
put : 0.62
on : 0.44


Line : 6669

to : 0.26
have : 0.40
provd : 0.82
m

In [26]:
#What's the TF-IDF of "Hamlet"?
#Print TF-IDF of "Hamlet" for each line where the terms appears
def find_tf_idf(word):
    w = word.lower()
    print(f"Tf-idf for {word}\n")
    for k, v in tf_idf_dict.items():
        if w in v:
            print(f"Line {k} : {v[w]:.2f}")

find_tf_idf('Hamlet')


Tf-idf for Hamlet

Line 0 : 0.39
Line 37 : 0.68
Line 38 : 0.45
Line 39 : 0.34
Line 40 : 0.30
Line 44 : 0.68
Line 269 : 0.30
Line 280 : 0.34
Line 388 : 0.39
Line 401 : 0.27
Line 406 : 0.34
Line 485 : 0.34
Line 487 : 2.70
Line 493 : 2.70
Line 497 : 0.39
Line 504 : 2.70
Line 511 : 2.70
Line 525 : 0.34
Line 558 : 0.34
Line 561 : 2.70
Line 567 : 0.39
Line 574 : 0.68
Line 576 : 2.70
Line 614 : 2.70
Line 622 : 2.70
Line 631 : 2.70
Line 638 : 2.70
Line 649 : 2.70
Line 656 : 2.70
Line 666 : 2.70
Line 672 : 2.70
Line 679 : 2.70
Line 685 : 2.70
Line 694 : 2.70
Line 716 : 2.70
Line 722 : 2.70
Line 734 : 2.70
Line 742 : 2.70
Line 749 : 2.70
Line 755 : 2.70
Line 761 : 2.70
Line 767 : 2.70
Line 773 : 2.70
Line 779 : 2.70
Line 785 : 2.70
Line 791 : 2.70
Line 803 : 2.70
Line 810 : 2.70
Line 817 : 2.70
Line 832 : 2.70
Line 858 : 0.34
Line 978 : 0.34
Line 1030 : 0.34
Line 1041 : 0.27
Line 1051 : 0.54
Line 1053 : 2.70
Line 1059 : 2.70
Line 1076 : 2.70
Line 1086 : 2.70
Line 1119 : 2.70
Line 1125 : 0.27
Lin

In [27]:
#What's the word with the highest TF-IDF in the book?

word_highest_tfidf = ''
highest_tfidf = 0

for k1, v1 in tf_idf_dict.items():
    for k2, v2 in v1.items():
        key_max = max(v1, key=v1.get)
        
    if highest_tfidf < v1[key_max]:
        word_highest_tfidf = key_max
        highest_tfidf = v1[key_max]
        
print(f"The word with the highest tf-idf is '{word_highest_tfidf}' where tf-idf is equal to {highest_tfidf}.")

The word with the highest tf-idf is 'writing' where tf-idf is equal to 8.1.



# Stretch Goal: Speaker count
Use a regular expression and looping over the hamlet.txt file to build a dictionary character_name -> # times speaking.

Who speaks the most often? Who speaks the least often?

In [28]:
import re

#Assume that the number of times speaking correspond to the number of cues for each of the characters to talk as oppose 
#to the number of lines or words spoken by each character.

with open(file, 'r', encoding = 'utf8') as hamlet:
    
    #Find the name of all the characters
    txt = hamlet.read()
    characters = set(re.findall('[A-Z]{3,}\.', txt)).difference({'III.', 'VII.', 'SCENE.', 'PROLOGUE.'})

    #Find number of cues for each characters
    speakers={}
    for char in characters: 
        speakers[char] = len(re.findall(char, txt))
    
    #Find the maximum and minimum number of cues for all characters
    max_key = max(speakers, key =speakers.get)
    min_key = min(speakers, key =speakers.get)
    max_times = speakers[max_key]
    min_times = speakers[min_key]        
    
    #Find the characters associated to the maximum and minimum number of cues and print results
    min_num_times = []
    max_num_times = []
    for k, v in speakers.items():
        if v == min_times:
            min_num_times.append(k)
        elif v == max_times:
            max_num_times.append(k)
            
    print(f"The character who speaks most often is: ")
    for name in max_num_times:
        print(name.title().replace('.', ''))
        
    print(f"\nThe characters who speak least often are: ")
    for name in min_num_times:
        print(name.title().replace('.', ''))       

The character who speaks most often is: 
Hamlet

The characters who speak least often are: 
Lucianus
Ambassador
Both
Lords
Servant
All
